##Read Bronze table

In [0]:

import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length



df = spark.table("workspace.bronze.erp_px_cat_g1v2")

In [0]:
df.display()

Failed to display Spark connect table, please contact Databricks support

##Silver Transformations

In [0]:
##trimimg

for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

In [0]:
##normalization maintenance flag to boolean

df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
     .when(F.upper(col("maintenance")) == "NO", F.lit(False))
     .otherwise(None)
)


In [0]:
##rename columns

RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)  

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_category")